In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
!python -m spacy download en_core_web_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 400.7 MB 6.3 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [15]:
import en_core_web_lg
import pandas as pd
import re
import random
import spacy
from spacy.util import minibatch, compounding
import warnings
import matplotlib.pyplot as plt

In [65]:
import json
 
with open('/content/TRAIN_FOOD_DATA(1).json') as json_file:
    TRAIN_FOOD_DATA = json.load(json_file)

In [66]:
with open('/content/TEST_FOOD_DATA(1).json') as json_file:
    TEST_FOOD_DATA = json.load(json_file)

In [67]:
TRAIN_FOOD_DATA

{'one_food': [['I had some dairy creamer at dinner last night',
   {'entities': [[11, 24, 'FOOD']]}],
  ['Last night, I ordered some white rice', {'entities': [[27, 37, 'FOOD']]}],
  ["I'm eating a crispbread", {'entities': [[13, 23, 'FOOD']]}],
  ["I'm done eating a rotelli", {'entities': [[18, 25, 'FOOD']]}],
  ['I ate my fioretti', {'entities': [[9, 17, 'FOOD']]}],
  ['This morning, I made a bowl of bun', {'entities': [[31, 34, 'FOOD']]}],
  ['I brought you some frittatas on my birthday',
   {'entities': [[19, 28, 'FOOD']]}],
  ['When I was having lunch I ate a besan', {'entities': [[32, 37, 'FOOD']]}],
  ['I brought a basket of peachy hearts for breakfast this morning',
   {'entities': [[22, 35, 'FOOD']]}],
  ['When I was having lunch I ate a low calorie dew',
   {'entities': [[32, 47, 'FOOD']]}],
  ['Last night, I ordered some star crunch cookies',
   {'entities': [[27, 46, 'FOOD']]}],
  ['I brought you some cherry pistachio oatmeal on my birthday',
   {'entities': [[19, 43, 'FOOD

In [68]:
TRAIN_DATA = TRAIN_FOOD_DATA["one_food"] + TRAIN_FOOD_DATA["two_foods"] + TRAIN_FOOD_DATA["three_foods"]

print("FOOD", len(TRAIN_DATA))


FOOD 501


In [69]:
TRAIN_DATA

[['I had some dairy creamer at dinner last night',
  {'entities': [[11, 24, 'FOOD']]}],
 ['Last night, I ordered some white rice', {'entities': [[27, 37, 'FOOD']]}],
 ["I'm eating a crispbread", {'entities': [[13, 23, 'FOOD']]}],
 ["I'm done eating a rotelli", {'entities': [[18, 25, 'FOOD']]}],
 ['I ate my fioretti', {'entities': [[9, 17, 'FOOD']]}],
 ['This morning, I made a bowl of bun', {'entities': [[31, 34, 'FOOD']]}],
 ['I brought you some frittatas on my birthday',
  {'entities': [[19, 28, 'FOOD']]}],
 ['When I was having lunch I ate a besan', {'entities': [[32, 37, 'FOOD']]}],
 ['I brought a basket of peachy hearts for breakfast this morning',
  {'entities': [[22, 35, 'FOOD']]}],
 ['When I was having lunch I ate a low calorie dew',
  {'entities': [[32, 47, 'FOOD']]}],
 ['Last night, I ordered some star crunch cookies',
  {'entities': [[27, 46, 'FOOD']]}],
 ['I brought you some cherry pistachio oatmeal on my birthday',
  {'entities': [[19, 43, 'FOOD']]}],
 ["I had papad yesterda

In [70]:
from spacy.training.example import Example

In [71]:
nlp = en_core_web_lg.load()

In [72]:
ner = nlp.get_pipe("ner")
ner.add_label("FOOD")

pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

epochs = 30
optimizer = nlp.resume_training()
with nlp.disable_pipes(*other_pipes), warnings.catch_warnings():
    warnings.filterwarnings("once", category=UserWarning, module='spacy')
    sizes = compounding(1.0, 4.0, 1.001)
    
    for epoch in range(epochs):
        examples = TRAIN_DATA
        random.shuffle(examples)
        batches = minibatch(examples, size=sizes)
        losses = {}

        for batch in batches:
            texts, annotations = zip(*batch)
            print(annotations)
            doc = nlp.make_doc(texts[0])
            example = Example.from_dict(doc, annotations[0])
            nlp.update([example], sgd=optimizer, drop=0.35, losses=losses)

        print("Losses ({}/{})".format(epoch + 1, epochs), losses)

({'entities': [[24, 30, 'FOOD']]},)
({'entities': [[27, 37, 'FOOD']]},)
({'entities': [[11, 24, 'FOOD']]},)
({'entities': [[9, 27, 'FOOD'], [29, 49, 'FOOD'], [54, 78, 'FOOD']]},)
({'entities': [[8, 16, 'FOOD'], [22, 29, 'FOOD']]},)
({'entities': [[8, 16, 'FOOD'], [20, 33, 'FOOD'], [38, 44, 'FOOD']]},)
({'entities': [[14, 25, 'FOOD'], [46, 65, 'FOOD']]},)
({'entities': [[19, 35, 'FOOD']]},)
({'entities': [[13, 23, 'FOOD']]},)
({'entities': [[21, 30, 'FOOD']]},)
({'entities': [[8, 17, 'FOOD'], [21, 32, 'FOOD'], [37, 43, 'FOOD']]},)
({'entities': [[14, 22, 'FOOD'], [29, 36, 'FOOD']]},)
({'entities': [[8, 25, 'FOOD'], [31, 47, 'FOOD'], [52, 58, 'FOOD']]},)
({'entities': [[20, 43, 'FOOD'], [45, 59, 'FOOD'], [64, 72, 'FOOD']]},)
({'entities': [[8, 14, 'FOOD'], [21, 29, 'FOOD']]},)
({'entities': [[13, 34, 'FOOD'], [36, 49, 'FOOD'], [54, 60, 'FOOD']]},)
({'entities': [[31, 42, 'FOOD']]},)
({'entities': [[16, 23, 'FOOD'], [29, 44, 'FOOD']]},)
({'entities': [[8, 14, 'FOOD'], [20, 36, 'FOOD']]},)

/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "Last night, I ate an organic mozzarella cheese wit..." with entities "[[21, 46, 'FOOD'], [52, 63, 'FOOD'], [68, 73, 'FOO...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,


({'entities': [[18, 27, 'FOOD']]},)
({'entities': [[9, 15, 'FOOD'], [17, 41, 'FOOD'], [46, 65, 'FOOD']]},)
({'entities': [[6, 30, 'FOOD'], [35, 41, 'FOOD']]},)
({'entities': [[8, 29, 'FOOD'], [36, 45, 'FOOD']]},)
({'entities': [[14, 21, 'FOOD'], [23, 31, 'FOOD'], [36, 42, 'FOOD']]},)
({'entities': [[13, 23, 'FOOD']]},)
({'entities': [[14, 23, 'FOOD'], [25, 42, 'FOOD'], [47, 60, 'FOOD']]},)
({'entities': [[14, 27, 'FOOD'], [29, 44, 'FOOD'], [49, 58, 'FOOD']]},)
({'entities': [[17, 25, 'FOOD']]},)
({'entities': [[8, 13, 'FOOD'], [17, 37, 'FOOD'], [42, 48, 'FOOD']]},)
({'entities': [[6, 14, 'FOOD']]},)
({'entities': [[8, 17, 'FOOD'], [23, 33, 'FOOD']]},)
({'entities': [[6, 26, 'FOOD'], [31, 40, 'FOOD']]},)
({'entities': [[14, 25, 'FOOD'], [32, 41, 'FOOD']]},)
({'entities': [[26, 37, 'FOOD'], [39, 54, 'FOOD'], [59, 81, 'FOOD']]},)
({'entities': [[8, 18, 'FOOD'], [25, 33, 'FOOD']]},)


/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "today I ate a ladyfingers and a lemonade  for lunc..." with entities "[[14, 25, 'FOOD'], [32, 41, 'FOOD']]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,


({'entities': [[13, 21, 'FOOD']]},)
({'entities': [[14, 26, 'FOOD'], [28, 35, 'FOOD'], [40, 48, 'FOOD']]},)
({'entities': [[13, 17, 'FOOD']]},)
({'entities': [[7, 12, 'FOOD']]},)
({'entities': [[8, 17, 'FOOD'], [24, 32, 'FOOD']]},)
({'entities': [[6, 20, 'FOOD']]},)
({'entities': [[9, 22, 'FOOD'], [27, 41, 'FOOD']]},)
({'entities': [[14, 26, 'FOOD'], [47, 57, 'FOOD']]},)
({'entities': [[16, 22, 'FOOD'], [28, 50, 'FOOD']]},)
({'entities': [[9, 30, 'FOOD'], [32, 50, 'FOOD'], [55, 67, 'FOOD']]},)
({'entities': [[14, 33, 'FOOD'], [54, 63, 'FOOD']]},)
({'entities': [[26, 35, 'FOOD'], [37, 40, 'FOOD'], [45, 51, 'FOOD']]},)
({'entities': [[21, 26, 'FOOD']]},)
({'entities': [[13, 21, 'FOOD']]},)
({'entities': [[8, 23, 'FOOD'], [27, 37, 'FOOD'], [42, 60, 'FOOD']]},)
({'entities': [[8, 15, 'FOOD'], [21, 33, 'FOOD'], [38, 47, 'FOOD']]},)
({'entities': [[26, 32, 'FOOD']]},)


...

({'entities': [[8, 16, 'FOOD'], [20, 42, 'FOOD'], [47, 53, 'FOOD']]},)
({'entities': [[14, 20, 'FOOD'], [22, 26, 'FOOD'], [31, 43, 'FOOD']]},)
({'entities': [[8, 30, 'FOOD'], [37, 44, 'FOOD']]},)
({'entities': [[27, 46, 'FOOD']]},)
({'entities': [[21, 28, 'FOOD'], [34, 54, 'FOOD'], [59, 69, 'FOOD']]},)
({'entities': [[14, 24, 'FOOD'], [31, 37, 'FOOD']]},)
({'entities': [[8, 19, 'FOOD'], [26, 43, 'FOOD']]},)
({'entities': [[13, 20, 'FOOD'], [22, 27, 'FOOD'], [32, 37, 'FOOD']]},)


/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "Last night, I ate an twists  with buttermilk panca..." with entities "[[21, 28, 'FOOD'], [34, 54, 'FOOD'], [59, 69, 'FOO...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,


({'entities': [[14, 31, 'FOOD'], [33, 42, 'FOOD'], [47, 51, 'FOOD']]},)
({'entities': [[13, 21, 'FOOD']]},)
({'entities': [[6, 12, 'FOOD']]},)
({'entities': [[14, 26, 'FOOD'], [47, 60, 'FOOD']]},)
({'entities': [[19, 29, 'FOOD']]},)
({'entities': [[20, 41, 'FOOD'], [43, 54, 'FOOD'], [59, 77, 'FOOD']]},)
({'entities': [[17, 23, 'FOOD']]},)
({'entities': [[14, 34, 'FOOD'], [36, 48, 'FOOD'], [53, 61, 'FOOD']]},)
({'entities': [[16, 30, 'FOOD'], [36, 50, 'FOOD']]},)
({'entities': [[6, 14, 'FOOD'], [19, 27, 'FOOD']]},)
({'entities': [[9, 15, 'FOOD'], [17, 21, 'FOOD'], [26, 37, 'FOOD']]},)
({'entities': [[15, 21, 'FOOD']]},)
({'entities': [[17, 32, 'FOOD']]},)
({'entities': [[26, 54, 'FOOD'], [56, 68, 'FOOD'], [73, 77, 'FOOD']]},)
({'entities': [[8, 18, 'FOOD'], [24, 29, 'FOOD']]},)
({'entities': [[14, 18, 'FOOD'], [39, 45, 'FOOD']]},)
({'entities': [[9, 19, 'FOOD']]},)
({'entities': [[8, 26, 'FOOD'], [30, 39, 'FOOD'], [44, 60, 'FOOD']]},)
({'entities': [[8, 15, 'FOOD'], [22, 31, 'FOOD']]},)

/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "today I ate a sorbet  and a rub for lunch" with entities "[[14, 21, 'FOOD'], [28, 31, 'FOOD']]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,


({'entities': [[8, 20, 'FOOD'], [27, 43, 'FOOD']]},)
({'entities': [[8, 29, 'FOOD'], [35, 48, 'FOOD']]},)
({'entities': [[21, 30, 'FOOD'], [36, 47, 'FOOD'], [52, 64, 'FOOD']]},)
({'entities': [[8, 32, 'FOOD'], [36, 46, 'FOOD'], [51, 60, 'FOOD']]},)
({'entities': [[21, 35, 'FOOD'], [41, 56, 'FOOD'], [61, 74, 'FOOD']]},)
({'entities': [[8, 21, 'FOOD'], [28, 36, 'FOOD']]},)
({'entities': [[8, 21, 'FOOD'], [25, 36, 'FOOD'], [41, 56, 'FOOD']]},)
({'entities': [[16, 33, 'FOOD']]},)
({'entities': [[16, 33, 'FOOD'], [39, 47, 'FOOD']]},)
({'entities': [[14, 25, 'FOOD'], [46, 69, 'FOOD']]},)
({'entities': [[9, 23, 'FOOD'], [28, 40, 'FOOD']]},)
({'entities': [[14, 24, 'FOOD'], [26, 35, 'FOOD'], [40, 59, 'FOOD']]},)
({'entities': [[11, 37, 'FOOD']]},)
Losses (1/30) {'ner': 576.6574614258233}
({'entities': [[14, 20, 'FOOD'], [22, 26, 'FOOD'], [31, 43, 'FOOD']]},)
({'entities': [[9, 18, 'FOOD'], [23, 33, 'FOOD']]},)
({'entities': [[9, 15, 'FOOD']]},)
({'entities': [[8, 13, 'FOOD']]},)
({'entities': 

In [73]:
from spacy import displacy

In [74]:
word = nlp("I ate Sushi yesterday. Maggi is a common fast food")

In [75]:
for ent in word.ents:
        print(ent.text, ent.label_)

Sushi FOOD
Maggi FOOD
common fast food FOOD


In [79]:
word = nlp("I eat Apple, Banana and Brade in every morning")
for ent in word.ents:
        print(ent.text, ent.label_)

Apple FOOD
Banana FOOD
Brade FOOD


In [80]:
# dictionary to hold our evaluation data
food_evaluation = {
    "one_food": {
        "correct": 0,
        "total": 0,
    },
    "two_foods": {
        "correct": 0,
        "total": 0
    },
    "three_foods": {
        "correct": 0,
        "total": 0
    }
}

word_evaluation = {
    "1_worded_foods": {
        "correct": 0,
        "total": 0
    },
    "2_worded_foods": {
        "correct": 0,
        "total": 0
    },
    "3_worded_foods": {
        "correct": 0,
        "total": 0
    }
}

for key in TEST_FOOD_DATA:
    foods = TEST_FOOD_DATA[key]

    for food in foods:
        sentence = food[0]
        entities = food[1]["entities"]

        for entity in entities:
            doc = nlp(sentence)
            correct_text = sentence[entity[0]:entity[1]]
            n_worded_food =  len(correct_text.split())

            for ent in doc.ents:
                if ent.label_ == entity[2] and ent.text == correct_text:
                    food_evaluation[key]["correct"] += 1
                    if n_worded_food > 0:
                        word_evaluation[f"{n_worded_food}_worded_foods"]["correct"] += 1
                    
                    break
            
            food_evaluation[key]["total"] += 1
            if n_worded_food > 0:
                word_evaluation[f"{n_worded_food}_worded_foods"]["total"] += 1

In [81]:
word_evaluation

{'1_worded_foods': {'correct': 967, 'total': 989},
 '2_worded_foods': {'correct': 654, 'total': 671},
 '3_worded_foods': {'correct': 537, 'total': 565}}

In [82]:
for key in word_evaluation:
    correct = word_evaluation[key]["correct"]
    total = word_evaluation[key]["total"]

    print(f"{key}: {correct / total * 100:.2f}%")

food_total_sum = 0
food_correct_sum = 0

print("---")
for key in food_evaluation:
    correct = food_evaluation[key]["correct"]
    total = food_evaluation[key]["total"]
    
    food_total_sum += total
    food_correct_sum += correct

    print(f"{key}: {correct / total * 100:.2f}%")

print(f"\nTotal: {food_correct_sum/food_total_sum * 100:.2f}%")

1_worded_foods: 97.78%
2_worded_foods: 97.47%
3_worded_foods: 95.04%
---
one_food: 96.31%
two_foods: 97.45%
three_foods: 97.64%

Total: 96.99%
